In [111]:
import pandas as pd
import numpy as np
import json
import re
from llms import gemini
from llms import chatGPT
from sklearn.metrics.pairwise import cosine_similarity
import seaborn as sns
from mpl_toolkits.mplot3d import Axes3D  # for 3D plotting
import plotly.express as px
from sklearn.manifold import TSNE
from reportlab.pdfgen import canvas
from reportlab.lib.pagesizes import letter
from reportlab.lib.utils import ImageReader
import plotly.io as pio
import requests
import pandas as pd
import aiohttp
import asyncio
import nest_asyncio
import os
from os import getenv
from dotenv import load_dotenv
import time
import ast 

load_dotenv("../.env",override=True)
GEO_KEY = getenv("GOOGLE_API_KEY")

x_chat = chatGPT()
x_gemini = gemini()


In [112]:
df = pd.read_csv("../output/results/bls_df.csv",index_col=0)
tsne = pd.read_csv("../output/results/tsne.csv",index_col=0)
df = pd.concat([df,tsne],axis=1)

# df.example_task_embedding = df.example_task_embedding.apply(lambda x: [float(y) for y in x.strip("[]").split(", ")])
# df.onet_task_embedding = df.onet_task_embedding.apply(lambda x: [float(y) for y in x.strip("[]").split(", ")])
# df.onet_title_embedding = df.onet_title_embedding.apply(lambda x: [float(y) for y in x.strip("[]").split(", ")])


In [175]:
df = df2

In [176]:
onet_occ = pd.read_csv("../input/onet/Occupation Data.csv")[["O*NET-SOC Code","Title"]]
onet_occ.columns = ["Detailed Occupation","onet_title"]
onet_occ["Detailed Occupation"] = onet_occ["Detailed Occupation"].apply(lambda x: x[:-3])

In [177]:
wage_employment = pd.read_csv("../input/wage_employment_2022/national2022.csv")
wage_employment = wage_employment[wage_employment.O_GROUP == "detailed"]
wage_employment = wage_employment.rename({"OCC_CODE": "Detailed Occupation"}, axis=1)
wage_employment = wage_employment.replace("#", np.nan).replace("*", np.nan).replace({",": ""}, regex=True)
wage_employment[["H_MEDIAN", "A_MEDIAN","TOT_EMP"]] = wage_employment[["H_MEDIAN", "A_MEDIAN","TOT_EMP"]].astype("float")
wage_employment["H_MEDIAN"] = wage_employment["H_MEDIAN"].round(0)
wage_employment = wage_employment[["Detailed Occupation","H_MEDIAN","A_MEDIAN","TOT_EMP"]]
wage_employment.columns = ["Detailed Occupation","H_MEDIAN_US","A_MEDIAN_US","TOT_EMP_US"]


all_tasks = df.groupby(["Task","Title"]).aggregate({"onet_weight":"sum"}).reset_index()
task_statements = pd.read_csv("../input/onet/Task Statements.csv")[["Task","Title"]]
all_tasks = all_tasks.merge(task_statements, on=["Task","Title"], how="outer")
all_tasks = all_tasks.merge(onet_occ, left_on="Title",right_on="onet_title").drop(columns=["Title"])

all_tasks = all_tasks.replace(np.nan,0)
perc_of_10K = (wage_employment.groupby("Detailed Occupation")["TOT_EMP_US"].first()/wage_employment["TOT_EMP_US"].sum()).reset_index() # num of people for every 10K workers
perc_of_10K = perc_of_10K.rename({"TOT_EMP_US":"Percent of 10K workers"},axis=1)


all_tasks = all_tasks.merge(perc_of_10K, on="Detailed Occupation")
ratios = all_tasks['onet_weight'] / all_tasks['Percent of 10K workers']
all_tasks["task_automation_rating"] = np.minimum(ratios, 1)


all_occupations = all_tasks.groupby("Detailed Occupation").aggregate({"task_automation_rating":"sum","Task":"count", "onet_title":"first"})
all_occupations.columns = ["occupation_# tasks automated","occupation_# of tasks","onet_title"]
all_occupations["occupation_automation_rating"] = all_occupations["occupation_# tasks automated"]/all_occupations["occupation_# of tasks"]
df = df.merge(all_tasks[["Task","onet_title","task_automation_rating","Percent of 10K workers"]], on=["Task","onet_title"], how="left")
df = df.merge(all_occupations,on=["Detailed Occupation","onet_title"],how="left")

df = df.merge(wage_employment, on="Detailed Occupation", how="left")
all_occupations = all_occupations.merge(wage_employment, on="Detailed Occupation")
all_tasks = all_tasks.merge(wage_employment, on="Detailed Occupation")
all_tasks = all_tasks.rename({"Title":"onet_title"},axis=1)

In [178]:
job_zones = pd.read_csv("../input/onet/Job Zones.csv",index_col=0)[["Title","Job Zone"]]
job_zones_ref = pd.read_csv("../input/onet/Job Zone Reference.csv",index_col=0)
df = df.merge(job_zones, on="Title", how="left")

In [179]:
education = pd.read_csv("../input/onet/Education, Training, and Experience.csv")[["Title","Data Value","Category", "Scale Name"]]
education["weighted_category"] = education["Data Value"]/100*education["Category"]
education = education.groupby(["Title","Scale Name"])["weighted_category"].sum().reset_index()
education = education.pivot(index=['Title'], columns='Scale Name', values='weighted_category').reset_index().drop(columns=["Importance"])
education.columns.name = None
education.columns = [col if not isinstance(col, tuple) else col[1] for col in education.columns]
df = df.merge(education, on="Title", how="left")

In [180]:
dwa = pd.read_csv("../input/onet/Tasks to DWAs.csv")[3:].reset_index(drop=True)
dwa = dwa[["DWA ID","DWA Title","Task"]]
# dwa = dwa.rename({"ID":"Task ID"},axis=1)
grouped = dwa.groupby("Task").aggregate({"DWA ID":"count"}).apply(lambda x: 1/x).reset_index()
grouped = grouped.rename({"DWA ID":"dwa_count"},axis=1)
dwa = dwa.merge(grouped,on="Task")
dwa_ref = pd.read_csv("../input/onet/DWA Reference.csv")[["Element Name","DWA ID"]]
dwa_ref = dwa_ref.rename({"Element Name":"activity"},axis=1)

df_activity = df.merge(dwa, on="Task",how="left")
df_activity = df_activity.merge(dwa_ref, on="DWA ID",how="left")
print(len(df_activity[df_activity["DWA ID"].isnull()]))

print(len(df_activity))

57
54079


In [188]:
all_tasks_activity = all_tasks.merge(dwa, on="Task",how="left")
all_tasks_activity = all_tasks_activity.merge(dwa_ref, on="DWA ID",how="left")

In [181]:
def parse_bls(wage_employment,type):
    wage_employment = wage_employment[wage_employment.O_GROUP == "detailed"]
    wage_employment = wage_employment.rename({"OCC_CODE": "Detailed Occupation"}, axis=1)
    wage_employment = wage_employment.replace("#", np.nan).replace(r"\*+", np.nan, regex=True).replace(",", "", regex=True)
    wage_employment[["H_MEDIAN", "A_MEDIAN","TOT_EMP","JOBS_1000"]] = wage_employment[["H_MEDIAN", "A_MEDIAN","TOT_EMP", "JOBS_1000"]].astype("float")
    wage_employment["H_MEDIAN"] = wage_employment["H_MEDIAN"].round(0)
    wage_employment = wage_employment[["AREA","Detailed Occupation","H_MEDIAN","A_MEDIAN","TOT_EMP","JOBS_1000"]]
    wage_employment.columns = ["AREA","Detailed Occupation","H_MEDIAN_"+type,"A_MEDIAN_"+type,"TOT_EMP_"+type, 'JOBS_1000_'+type]
    return wage_employment


nonmetro = pd.read_csv("../input/wage_employment_2022/rural2022.csv")
metro = pd.read_csv("../input/wage_employment_2022/metro2022.csv")
nonmetro = parse_bls(nonmetro,"RURAL")
metro = parse_bls(metro,"METRO")

#figures out how much to weight each area
def calculate_weights(df,type):
    grouped = df.groupby("Detailed Occupation")["TOT_EMP_"+type].sum().reset_index()
    grouped = grouped.rename({"TOT_EMP_"+type: "sum_"+type}, axis=1)
    df = df.merge(grouped, on="Detailed Occupation")
    df["weight_area_"+type] = df["TOT_EMP_"+type] / df["sum_"+type]
    df["weighted_JOBS_1000_"+type] = df["JOBS_1000_"+type]*df["weight_area_"+type]
    return df
metro = calculate_weights(metro,"METRO")
nonmetro = calculate_weights(nonmetro,"RURAL")

metro_grouped = metro.groupby("Detailed Occupation").aggregate({"H_MEDIAN_METRO":"mean","TOT_EMP_METRO":"sum","A_MEDIAN_METRO":"mean", "weighted_JOBS_1000_METRO":"sum"},axis=1).reset_index()
nonmetro_grouped = nonmetro.groupby("Detailed Occupation").aggregate({"H_MEDIAN_RURAL":"mean","TOT_EMP_RURAL":"sum","A_MEDIAN_RURAL":"mean","weighted_JOBS_1000_RURAL":"sum"},axis=1).reset_index()

df = df.merge(metro_grouped,on="Detailed Occupation",how="left")
df = df.merge(nonmetro_grouped,on="Detailed Occupation",how="left")

In [182]:
metro.columns = [col.replace("_METRO","") for col in metro.columns]
metro["Type"] = "METRO"
nonmetro.columns = [col.replace("_RURAL","") for col in nonmetro.columns]
nonmetro["Type"] = "RURAL"
locations = pd.concat([metro, nonmetro],axis=0)
locations = locations.merge(all_occupations,on="Detailed Occupation")
locations["weighted_automation_#_per_1000_people"] = locations["JOBS_1000"]*locations["occupation_automation_rating"]
locations = locations.groupby("AREA").aggregate({"weighted_automation_#_per_1000_people":"sum", "Type":"first"}).reset_index()


geo = pd.read_csv("../output/parsed_BLS_data/geography.csv",index_col=0)
geo.Boundary = geo.Boundary.apply(lambda x: ast.literal_eval(x) if not pd.isnull(x) else x)
geo = geo[~geo["name"].str.contains("Census Area")]


locations = locations.merge(geo, on="AREA",how="left")
locations = locations.dropna(subset=["name"])

locations.to_csv("../output/EDA_data/automation_by_geography.csv")

In [185]:
all_occupations.to_csv("../output/EDA_data/all_occupations_automation.csv")
all_tasks.to_csv("../output/EDA_data/all_tasks_automation.csv")
df.to_csv("../output/EDA_data/startup_data.csv")
df_activity.to_csv("../output/EDA_data/startup_data_with_activities.csv")

In [189]:
all_tasks_activity.to_csv("../output/EDA_data/all_tasks_automation_activity.csv")